In [62]:
import os
import os.path
import sys
import copy
import string
import numpy
import sklearn.ensemble
import sklearn.model_selection 
import sklearn.pipeline 

from gensim.models.word2vec import Word2Vec

from multiprocessing import cpu_count

from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

from collections import namedtuple



In [63]:
input_folder_name = "C:\\bbc_insight" 
categories = ["business", "entertainment", "politics", "sport", "tech"] 


# Word2Vec number of features
num_features = 300
# Newsline folder and format
data_folder = 'C:\\semdoc\\data\\bbc\\'


start_alpha=0.01

infer_epoch=511

vector_size = num_features

window_size = 15

min_count = 1

sampling_threshold = 1e-5

negative_size = 5

train_epoch = 100

dm = 0 #0 = dbow; 1 = dmpv

worker_count = 1 #number of parallel processes

documents = []
labels = []
# For doc2vec.
tagged_documents = []
for i in range(len(categories)):
    category_name = categories[i]#categories[i].upper()
    folder_name = os.path.join(input_folder_name, category_name)
    base_filenames = os.listdir(folder_name)
    for j in range(len(base_filenames)):
        base_filename = base_filenames[j]
        filename = os.path.join(folder_name, base_filename)
        
        with open(filename, "r") as ifstr:
            text = ifstr.read()
        words = text.split()
        documents.append(words)
        tagged_document = TaggedDocument(words, [i])
        tagged_documents.append(tagged_document)
        labels.append(i)        
    
    

In [64]:
#doc2vec kısmı 
#d2v_model = Doc2Vec(alpha=0.025, min_alpha=0.025)  # use fixed learning rate
#d2v_model.build_vocab(sentences)
#for epoch in range(10):
#    d2v_model.train(sentences)
#    d2v_model.alpha -= 0.002  # decrease the learning rate
#    d2v_model.min_alpha = d2v_model.alpha  # fix the learning rate, no decay
d2v_model = Doc2Vec(tagged_documents, size=vector_size, window=window_size, min_count=min_count, sample=sampling_threshold, workers=worker_count, hs=0, dm=dm, negative=negative_size, dbow_words=1, dm_concat=1,  iter=train_epoch)
d2v_model.save(data_folder + 'bbc_insight.doc2vec')

#d2v_model = Doc2Vec(size = num_features, alpha=0.025, min_alpha=0.025)  # use fixed learning rate
#d2v_model.build_vocab(tagged_documents)

C:\ProgramData\Anaconda3\lib\site-packages\gensim-3.3.0-py3.6-win-amd64.egg\gensim\models\doc2vec.py:355: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
C:\ProgramData\Anaconda3\lib\site-packages\gensim-3.3.0-py3.6-win-amd64.egg\gensim\models\doc2vec.py:359: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [66]:
# Vectorize each document
num_categories = len(categories)
number_of_documents = len(documents)
X = numpy.zeros(shape=(number_of_documents, num_features)).astype(numpy.float32)
Y = numpy.zeros(shape=(number_of_documents, 1)).astype(numpy.float32)

empty_word = numpy.zeros(num_features).astype(numpy.float32)
for i in range(number_of_documents):
    doc = documents[i]
    vec = d2v_model.infer_vector(doc, alpha=start_alpha, steps=infer_epoch)
    if i == 1:
        print(vec) 
    X[i, :] = numpy.array(vec)
    Y[i, :] = labels[i]

[ 1.00580908e-01 -1.85163729e-02 -1.28635645e-01 -2.07279921e-01
  3.23981076e-01 -8.99015144e-02  2.01788083e-01  4.17139195e-02
 -5.41324243e-02 -2.17137590e-01  5.38682304e-02  2.95345411e-02
 -1.46104962e-01 -4.58891168e-02 -4.51719500e-02 -2.54812241e-01
  2.54250821e-02  2.52736896e-01  1.68476850e-01 -5.73144816e-02
 -2.49788359e-01 -2.22860053e-01 -1.83388725e-01 -6.85273781e-02
 -2.14069769e-01  7.66467601e-02  4.80848029e-02  8.88333842e-02
  9.23575386e-02 -7.04932436e-02  9.85810906e-03  1.45978034e-01
  1.31680518e-01 -2.64448106e-01  3.53234231e-01  1.74500316e-01
  4.74480033e-01 -2.48010322e-01  3.41401041e-01 -2.47960649e-02
  2.98951328e-01  4.37813960e-02 -4.20332141e-03 -2.47844636e-01
 -3.04268718e-01  1.85850993e-01 -9.60609764e-02 -2.34943524e-01
  8.93947855e-02  2.34043062e-01 -5.96826430e-03 -2.75080115e-01
  4.01240177e-02 -2.16789261e-01 -1.77443922e-01 -3.73681374e-02
  2.02108964e-01  3.48654985e-01  1.17694467e-01  1.81725752e-02
  3.27359378e-01 -7.22914

In [67]:
# Gradient Boosting
#model.fit(X, Y)
ctg = sklearn.ensemble.GradientBoostingClassifier()
model_selection_mdl = sklearn.model_selection
pipeline_mdl = sklearn.pipeline
pipeline_list = [("categorizer", ctg)]
pipeline = pipeline_mdl.Pipeline(pipeline_list)
param_grid = [
    {

        'categorizer__n_estimators': [63, 127, 255, 511],
        'categorizer__learning_rate':[0.1, 0.01, 0.001],
    }
]

grid_searcher = model_selection_mdl.GridSearchCV(pipeline, cv=3, n_jobs=1, param_grid=param_grid, verbose =3)
grid_searcher.fit(X, Y)
#scores = model_selection_mdl.cross_val_score(model, X, Y, cv=5)
#accuracy = scores.mean()

accuracy = grid_searcher.best_score_
print("Accuracy: %.3f" % (accuracy))

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] categorizer__learning_rate=0.1, categorizer__n_estimators=63 ....


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV]  categorizer__learning_rate=0.1, categorizer__n_estimators=63, score=0.8694481830417228, total=  19.1s
[CV] categorizer__learning_rate=0.1, categorizer__n_estimators=63 ....


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.1s remaining:    0.0s
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV]  categorizer__learning_rate=0.1, categorizer__n_estimators=63, score=0.876010781671159, total=  18.6s
[CV] categorizer__learning_rate=0.1, categorizer__n_estimators=63 ....


...

[CV]  categorizer__learning_rate=0.001, categorizer__n_estimators=511, score=0.6401617250673854, total= 2.1min
[CV] categorizer__learning_rate=0.001, categorizer__n_estimators=511 .


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV]  categorizer__learning_rate=0.001, categorizer__n_estimators=511, score=0.7337837837837838, total= 2.1min


[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed: 37.8min finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.918
